# Lectura de datos

In [1]:
import os

ruta_al_directorio = '/home/TFM/code'

os.chdir(ruta_al_directorio)

In [2]:
import torch
import numpy as np
import pandas as pd
import random
from gnn.model import get_gnnNets
from gendata import get_dataset
from utils.parser_utils import (
    arg_parse,
    create_args_group,
    fix_random_seed,
    get_data_args,
    get_graph_size_args,
)
from pathlib import Path
from torch_geometric.utils import degree


## Forma manual - entender el código proporcionado

In [3]:
import os.path as osp
import torch
import mat73
from sklearn.model_selection import train_test_split
import os
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
import torch
from torch_geometric.data import Data
import numpy as np
from utils.gen_utils import from_edge_index_to_adj, padded_datalist

In [4]:
names = {
        "uk": ["uk", "Uk", "UK", None],
        "ieee24": ["ieee24", "Ieee24", "IEEE24", None],
        "ieee39": ["ieee39", "Ieee39", "IEEE39", None],
        "ieee118": ["ieee118", "Ieee118", "IEEE118", None],
            }
name = "ieee24"
raw_dir = '/home/dataset/{}/raw'.format(name)

In [5]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [6]:
# load branch list also called edge order or edge index
path = os.path.join(raw_dir, 'blist.mat')
edge_order = mat73.loadmat(path)
edge_order = torch.tensor(edge_order["bList"] - 1)


# load output binary classification labels
path = os.path.join(raw_dir, 'of_bi.mat')
of_bi = mat73.loadmat(path)

# load output binary regression labels
path = os.path.join(raw_dir, 'of_reg.mat')
of_reg = mat73.loadmat(path)

# load output mc labels
path = os.path.join(raw_dir, 'of_mc.mat')
of_mc = mat73.loadmat(path)


# load output node feature matrix
path = os.path.join(raw_dir, 'Bf.mat')
node_f = mat73.loadmat(path)


# load output edge feature matrix
path = os.path.join(raw_dir, 'Ef.mat')
edge_f = mat73.loadmat(path)


# load explanations
path = os.path.join(raw_dir, "exp.mat")
exp = mat73.loadmat(path)

In [7]:
node_f = node_f['B_f_tot']
edge_f = edge_f['E_f_post']
of_bi = of_bi['output_features']
of_mc = of_mc['category']
of_reg = of_reg['dns_MW']
exp_mask = exp["explainations"]

In [8]:
data_list = []
adj_list = []
max_num_nodes = 0
index = 0


Bucle principal de procesamiento:

Para cada grafo del dataset (resultado de un corte), hacemos.
- Cogemos las características de los nodos y aristas
- Procesamos las aristas:
    1. Se crea una máscara que tiene a 1 las aristas que fueron parte del inicio del fallo
    2. Se eliminan las contingencias (aristas que tengan todo a 0 al final) de la máscara, y se duplica la máscara 
    3. Se eliminan las contingencias de la matriz de atributos de aristas
    4. Se duplican las características de las aristas, para tener en cuenta los ejes en ambos sentidos
    5. Se eliminan las contingencias de la lista de aristas general (para obtener una versión de cada grafo)
    6. Se duplica la lista de aristas general
- Creamos un objeto Data, que representa un grafo, y lo procesamos correctamente:
    - Las etiquetas 0 se transforman a 1, y las etiquetas 1 se cambian a -1
    - Se guarda en una lista de objetos Data
- Se obtiene la matriz de adyacencia, y se guarda en una lista de matrices de adyacencia




In [9]:
def th_delete(tensor, indices):
            mask = torch.ones(tensor.size(), dtype=torch.bool)
            mask[indices] = False
            return tensor[mask]

In [10]:
datatype = 'binary'
pre_filter = None
pre_transform = None
#Iteramos a través de todo el dataset:

for i in range(len(node_f)):
    # node output features
    x = torch.tensor(node_f[i][0], dtype=torch.float32).reshape([-1, 3]).to(device)
    # edge output features
    f = torch.tensor(edge_f[i][0], dtype=torch.float32)


    #Se ponen a 1 las aristas que empezaron con la cascada de fallos
    e_mask = torch.zeros(len(edge_f[i][0]), 1)
    if exp_mask[i][0] is None:  # .all() == 0:
        e_mask = e_mask
    else:
        e_mask[exp_mask[i][0].astype('int')-1] = 1

        
    # contigency lists, finds where do we have contigencies from the .mat edge feature matrices
    # ( if a line is part of the contigency list all egde features are set 0)
    cont = [j for j in range(len(f)) if np.all(np.array(f[j])) == 0]
    e_mask_post = th_delete(e_mask, cont)
    e_mask_post = torch.cat((e_mask_post, e_mask_post), 0).to(device)

    
    # remove edge features of the associated line
    f_tot = th_delete(f, cont).reshape([-1, 4]).type(torch.float32)


    # concat the post-contigency edge feature matrix to take into account the reversed edges
    f_totw = torch.cat((f_tot, f_tot), 0).to(device)

    
    # remove failed lines from branch list
    edge_iw = th_delete(edge_order, cont).reshape(-1, 2).type(torch.long)
    # flip branch list
    edge_iwr = torch.fliplr(edge_iw)
    
    #  and concat the non flipped and flipped branch list
    edge_iw = torch.cat((edge_iw, edge_iwr), 0)
    edge_iw = edge_iw.t().contiguous().to(device)

    if datatype.lower() == 'binary':
        ydata = torch.tensor(of_bi[i][0], dtype=torch.float, device=device).view(1, -1)
    if datatype.lower() == 'regression':
        ydata = torch.tensor(of_reg[i], dtype=torch.float, device=device).view(1, -1)
    if datatype.lower() == 'multiclass':
        #do argmax
        ydata = torch.tensor(np.argmax(of_mc[i][0]), dtype=torch.float, device=device).view(1, -1)
        # ydata = torch.tensor(of_mc[i][0], dtype=torch.int, device=device).view(1, -1)
    # Fill Data object, 1 Data object -> 1 graph

    data = Data(x=x, edge_index=edge_iw, edge_attr=f_totw, y=ydata, edge_mask=e_mask_post, idx=index)
    index+=1
    if ydata == 0:
        ydata_cf = torch.tensor(1, dtype=torch.int, device=device).view(-1)
    else:
        ydata_cf = torch.tensor(-1, dtype=torch.int, device=device).view(-1)
    data.y_cf = ydata_cf


    
    adj = from_edge_index_to_adj(data.edge_index, None, data.num_nodes)
    adj_list.append(adj)
    max_num_nodes = max(max_num_nodes, data.num_nodes)
    # append Data object to datalist
    data_list.append(data)
    if pre_filter is not None and not pre_filter(data):
        continue

    if pre_transform is not None:
        data = pre_transform(data)




data_list = padded_datalist(data_list, adj_list, max_num_nodes)


In [11]:
grafo_1 = data_list[0]
grafo_1

Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=0, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3])

In [19]:
data_list

[Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=0, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=1, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=2, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=3, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=4, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=5, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index=[2, 74], edge_attr=[74, 4], y=[1, 1], edge_mask=[74], idx=6, y_cf=[1], adj_padded=[24, 24], x_padded=[24, 3]),
 Data(x=[24, 3], edge_index

## Automático 

Lo hacemos con la librería adecuada - InMemoryDataset es una clase proporcionada por PyTorch Geometric, una biblioteca diseñada para trabajar con datos de grafos en PyTorch. Esta clase es una subclase de torch_geometric.data.Dataset y está diseñada para manejar conjuntos de datos de grafos que se pueden cargar completamente en la memoria RAM.

El código correspondiente a la clase InMemoryDataset viene proporcionado:

In [12]:
from dataset.powergrid import PowerGrid

In [13]:
def get_dataset(dataset_root, **params):

    dataset_name = params["dataset_name"]
    datatype = params["datatype"]
    print(f"Loading {dataset_name} dataset...")

    if dataset_name.lower() in list(PowerGrid.names.keys()):
        return PowerGrid(root=dataset_root, name=dataset_name, datatype=datatype)
    
    # NO VAMOS A USAR DATASETS SINTÉTICOS
    #elif dataset_name.lower() in list(SynGraphDataset.names.keys()):
    #   dataset = SynGraphDataset(root=dataset_root, name=dataset_name, **kwargs)
    #   return dataset
    else:
        raise ValueError(f"{dataset_name} is not defined.")

In [14]:
CKPT_ROOT = "/home/TFM/code/extras/"

DATA_DIR = "/home/dataset/"
MODEL_DIR = CKPT_ROOT + "model/"
LOG_DIR = CKPT_ROOT + "logs/"
RESULT_DIR = CKPT_ROOT + "results/"
MASK_DIR = CKPT_ROOT + "mask/"


In [15]:
dataset_params = {
    "dataset_name": "ieee24",
    "random_seed": 0,
    "datatype": "classification",
    "train_ratio": 0.8,
    "val_ratio": 0.15,
    "test_ratio": 0.1
}

args = {
    'data_save_dir': DATA_DIR,
    'dest': '/home/TFM/',
    'logs_save_dir': LOG_DIR,
    'mask_save_dir': MASK_DIR,
    'model_save_dir': MODEL_DIR,
    'result_save_dir': RESULT_DIR,

}

In [16]:


dataset = get_dataset(args['data_save_dir'], **dataset_params)





Loading ieee24 dataset...


In [17]:
dataset

PowerGrid(12900)

In [18]:
dataset.data.x = dataset.data.x.float()
dataset.data.y = dataset.data.y.squeeze().long()


/usr/local/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


### Visualización


In [22]:
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 17.5 MB/s eta 0:00:0000:0100:01


In [34]:

import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
def show_graph(data):
    G = nx.Graph()
    G.add_edges_from(data.edge_index.t().tolist())  # Convertir edge_index a una lista de tuplas

    for i, x in enumerate(data.x.tolist()):
        G.nodes[i].update({'Net active power': x[0], 'Net apparent power': x[1], 'Voltage magnitude': x[2]})

    for i, (u, v) in enumerate(data.edge_index.t().tolist()):
        G[u][v].update({'Active power flow': data.edge_attr[i].tolist()[0], 
                        'Reactive power flow': data.edge_attr[i].tolist()[1], 
                        'Line reactance': data.edge_attr[i].tolist()[2], 
                        'Line rating': data.edge_attr[i].tolist()[3]})

    pos = nx.spring_layout(G)  # Calcular las posiciones de los nodos

    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    node_degrees = dict(G.degree())
    node_colors = [degree for node, degree in node_degrees.items()]


    node_text = []
    for node in G.nodes():
        attributes = G.nodes[node]
        text = f'Node {node}<br>'
        for key, value in attributes.items():
            text += f'{key}: {value}<br>'
        node_text.append(text)


    edge_text = []
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        attributes = G[edge[0]][edge[1]]
        text = f'Edge {edge}<br>'
        for key, value in attributes.items():
            text += f'{key}: {value}<br>'
        edge_text.append(text)


    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_text,  # Mostrar información adicional al pasar el cursor sobre los nodos
        marker=dict(
            color=node_colors,
            showscale=True,
            colorscale='YlGnBu',
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        text=edge_text,
        mode='lines')

    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Red eléctrica',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()


In [43]:
show_graph(dataset[1])

In [44]:
show_graph(dataset[0])